In [1]:
import json
import pandas as pd
import json
import regex
import pprint
def preprocess_mafalda(input_paths):
    label_map = {
        # Fallacy of Credibility
        'ad hominem': 'ad hominem',
        'tu quoque': 'tu quoque',
        'guilt by association': 'guilt by association',
        'ad populum': 'ad populum',
        'appeal to nature': 'appeal to nature',
        'appeal to tradition': 'appeal to tradition',
        'appeal to (false) authority': 'appeal to false authority',
        # Fallacy of Logic
        'causal oversimplification': 'causal oversimplification',
        'hasty generalization': 'hasty generalization',
        'false causality': 'false causality',
        'false analogy': 'false analogy',
        'false dilemma': 'false dilemma',
        'slippery slope': 'slippery slope',
        'fallacy of division': 'fallacy of division',
        'straw man': 'straw man',
        'circular reasoning': 'circular reasoning',
        'equivocation': 'equivocation',
        # Fallacy of Emotion
        'appeal to positive emotion': 'appeal to positive emotion',
        'appeal to anger': 'appeal to anger',
        'appeal to fear': 'appeal to fear',
        'appeal to pity': 'appeal to pity',
        'Appeal to Ridicule': 'appeal to ridicule',
        'appeal to ridicule': 'appeal to ridicule',
        'appeal to worse problems': 'appeal to worse problems',
        # Others
        'nothing': 'no fallacy',
        'to clean': 'to be confirmed',
    }
    new_data = []
    all_labels = []
    label_counts = []
    for input_path in input_paths:
        for i, one_data in enumerate([json.loads(line) for line in open(input_path)]):
            # label as dominant label
            labels = [label_map[l[2]] for l in one_data['labels']]
            if len(labels) == 0:
                labels = ['no fallacy']
            if len(labels) > 1:
                labels = [l for l in labels if l != 'no fallacy']
            segment_annotations = [{'segment':seg, 'labels':[l[0] for l in lb_ls]} for seg, lb_ls in json.loads(one_data['sentences_with_labels']).items()]

            if len(labels) == 1:
                other_labels = []
            else:
                # find the dominant label
                label_count = {}
                for item in segment_annotations:
                    s, ls = item['segment'], item['labels']
                    for l in ls:
                        if l not in ['nothing', 'to clean']:
                            if l not in label_count:
                                label_count[l] = 1
                            else:
                                label_count[l] += 1
                max_count = max(list(label_count.values()))
                dominant_labels = [label_map[k] for k, v in label_count.items() if v == max_count] # most dominant 
                other_labels = list(set(labels) - set(dominant_labels))
                labels = dominant_labels

            if len(other_labels) == 0:
                other_labels = [""]
            one_new_data = {"id": i+1, 'text': one_data['text'].replace("\n"," ").strip(), 'label': labels, 'other_labels': other_labels,
                            "segment_annotation": segment_annotations,
                            "comment": [{'fallacy': cmt.split(":")[0].strip(), 'comment': ":".join(cmt.split(":")[1:]).strip()}  for cmt in one_data['comments']]
                        }
            new_data.append(one_new_data)
            all_labels.extend(labels)
            label_counts.append(len(labels))

    fal_list = list(set(all_labels))
    fal_count = {}
    for l in all_labels:
        if l not in fal_count:
            fal_count[l] = 1
        else:
            fal_count[l] += 1

    label_counts_dist = {}
    for c in label_counts:
        if c not in label_counts_dist:
            label_counts_dist[c] = 1
        else:
            label_counts_dist[c] += 1
    
    print(f"Test: A total of {len(new_data)} examples.")
    print(pprint.pformat(fal_count))
    print(pprint.pformat(label_counts_dist))
    json.dump(new_data, open('test.json', 'w'), indent=4)
    
    return
preprocess_mafalda(['./raw/gold_standard_dataset.jsonl'])


Test: A total of 200 examples.
{'ad hominem': 7,
 'ad populum': 12,
 'appeal to anger': 3,
 'appeal to false authority': 6,
 'appeal to fear': 5,
 'appeal to nature': 9,
 'appeal to positive emotion': 2,
 'appeal to ridicule': 7,
 'appeal to tradition': 3,
 'appeal to worse problems': 5,
 'causal oversimplification': 16,
 'circular reasoning': 8,
 'equivocation': 4,
 'fallacy of division': 2,
 'false analogy': 5,
 'false causality': 12,
 'false dilemma': 6,
 'guilt by association': 3,
 'hasty generalization': 20,
 'no fallacy': 63,
 'slippery slope': 9,
 'straw man': 2,
 'tu quoque': 3}
{1: 190, 2: 8, 3: 2}
